In [1]:
!python -m pip install --user --upgrade pip

     |████████████████████████████████| 1.5MB 4.1MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 4.0 MB/s 
     |████████████████████████████████| 636 kB 6.3 MB/s 
     |████████████████████████████████| 1.5 MB 1.7 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 4.8 MB/s 
     |████████████████████████████████| 1.0 MB 18.4 MB/s 
     |████████████████████████████████| 61 kB 6.5 MB/s 
  Created wheel for kfp: filename=kfp-1.4.0-py3-none-any.whl size=221064 sha256=ed0bbf314f97ae3705126c0d39dcb25feec160d8a3375a8eb0924c5fcca8513f
  Stored in directory: /root/.cache

In [66]:
import kfp
from kfp import dsl
import kfp.components as comp

In [67]:
def load_data_op():
    return dsl.ContainerOp(
        name = 'Load Data',
        image = 'mavencodevv/load_gb:v.0.1',
        arguments = [],
        file_outputs={
            'data': '/load_data/data'
        }      
    )

In [68]:
def preprocess_op(data):
    return dsl.ContainerOp(
        name = 'Preprocess Data',
        image = 'mavencodevv/preprocess_gb:v.0.2',
        arguments = ['--data', data],
        file_outputs={
            'clean_data': '/preprocess/clean_data'   
        }
    )

In [100]:
def rf_op(clean_data):
    return dsl.ContainerOp(
        name = 'Randomforest',
        image = 'mavencodevv/randomforest_gb:v.0.2',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'rf_metrics': '/randomforest/rf_metrics' 
        }
    )

In [101]:
def cb_op(clean_data):
    return dsl.ContainerOp(
        name = 'CatBoost',
        image = 'mavencodevv/catboost_gb:v.0.4',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'cb_metrics': '/catboost/cb_metrics' 
        }
    )

In [102]:
def gbc_op(clean_data):
    return dsl.ContainerOp(
        name = 'GBC',
        image = 'mavencodevv/gbclassifier_gb:v.0.1',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'gbc_metrics': '/gbc/gbc_metrics' 
        }
    )

In [103]:
def lr_op(clean_data):
    return dsl.ContainerOp(
        name = 'Logistic Regression',
        image = 'mavencodevv/logistic_gb:v.0.1',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'lr_metrics': '/lr/lr_metrics' 
        }
    )

In [115]:
def results(rf_metrics,cb_metrics,gbc_metrics,lr_metrics):
    return dsl.ContainerOp(
        name = 'Results',
        image = 'mavencodevv/results_gb:v.0.3',
        arguments = ['--rf_metrics', rf_metrics,
                     '--cb_metrics', cb_metrics,
                     '--gbc_metrics', gbc_metrics,
                     '--lr_metrics', lr_metrics
                     ] 
        
    )

In [119]:
@dsl.pipeline(
    name='Fault Diagnosis of a Gearbox',
   description='An ML reusable pipeline that predicts whether gearbox sensor data is healthy or broken'
)

# Define parameters to be fed into pipeline
def gb_pipeline(): 
    
    _load_data_op = load_data_op()
    
    _preprocess_op = preprocess_op(
        dsl.InputArgumentPath(_load_data_op.outputs['data'])).after(_load_data_op)
    
    _rf_op = rf_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)

    _cb_op = cb_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
    _gbc_op = gbc_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
    _lr_op = lr_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
   
    _results_op = results(
        dsl.InputArgumentPath(_rf_op.outputs['rf_metrics']),
        dsl.InputArgumentPath(_cb_op.outputs['cb_metrics']),
        dsl.InputArgumentPath(_gbc_op.outputs['gbc_metrics']),
        dsl.InputArgumentPath(_lr_op.outputs['lr_metrics'])).after(_rf_op,_cb_op,_gbc_op,_lr_op)
    

In [ ]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'gearbox_diagnosis_pipeline2'

kfp.compiler.Compiler().compile(gb_pipeline, '{}.yaml'.format(experiment_name))

In [ ]:
client = kfp.Client()
client.create_run_from_pipeline_func(gb_pipeline, arguments={})